# CatBoost Implementation

In [ ]:
!pip install catboost==1.2.3
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: catboost
    Found existing installation: catboost 1.2
    Uninstalling catboost-1.2:
      Successfully uninstalled catboost-1.2
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
!pip install numpy==1.22.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 83.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for numpy: filename=numpy-1.22.4-cp311-cp311-linux_x86_64.whl size=17328447 sha256=8388db9859dd00164ed054841e1a0b51f7783f86e05cc9bcf071415afee4f07c
  Stored in directory: /root/.cache/pip/wheels/8e/c0/7e/1583fa989ccf57e2059824c8783691f4927f2ce7b77cec9da2
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mizani 0.13.1 requires numpy>=1.23.5, but you have numpy 1.22.4 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.22.4 which i

In [ ]:
#!/bin/bash
!curl -L -o ~/beth-dataset.zip\
  https://www.kaggle.com/api/v1/datasets/download/katehighnam/beth-dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 39.7M  100 39.7M    0     0  38.5M      0  0:00:01  0:00:01 --:--:--  173M


In [ ]:
!unzip /root/beth-dataset.zip

Archive:  /root/beth-dataset.zip
  inflating: labelled_2021may-ip-10-100-1-105-dns.csv  
  inflating: labelled_2021may-ip-10-100-1-105.csv  
  inflating: labelled_2021may-ip-10-100-1-186-dns.csv  
  inflating: labelled_2021may-ip-10-100-1-186.csv  
  inflating: labelled_2021may-ip-10-100-1-26-dns.csv  
  inflating: labelled_2021may-ip-10-100-1-26.csv  
  inflating: labelled_2021may-ip-10-100-1-4-dns.csv  
  inflating: labelled_2021may-ip-10-100-1-4.csv  
  inflating: labelled_2021may-ip-10-100-1-95-dns.csv  
  inflating: labelled_2021may-ip-10-100-1-95.csv  
  inflating: labelled_2021may-ubuntu-dns.csv  
  inflating: labelled_2021may-ubuntu.csv  
  inflating: labelled_testing_data.csv  
  inflating: labelled_training_data.csv  
  inflating: labelled_validation_data.csv  


In [ ]:
import pandas as pd
import os

csv_files = [
    'labelled_testing_data.csv',
    'labelled_training_data.csv',
    'labelled_validation_data.csv'
]
combined_data = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
combined_data.to_csv('combined_dataset.csv', index=False)

print(f"Combined dataset shape: {combined_data.shape}")
print(combined_data.head())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('combined_dataset.csv')

def auto_classify_columns(data):
    categorical_columns = []
    numerical_columns = []

    for col in data.columns:
        if data[col].dtype == 'object':
            unique_vals = data[col].nunique()
            if unique_vals < 0.1 * len(data):
                categorical_columns.append(col)
            else:
                try:
                    data[col] = pd.to_numeric(data[col], errors='raise')
                    numerical_columns.append(col)
                except:
                    categorical_columns.append(col)
        else:
            numerical_columns.append(col)

    return categorical_columns, numerical_columns

categorical_columns, numerical_columns = auto_classify_columns(data)

print("Categorical columns:", categorical_columns)
print("Numerical columns:", numerical_columns)

Combined dataset shape: (1141078, 16)
    timestamp  processId  threadId  parentProcessId  userId  mountNamespace  \
0  129.050634        382       382                1     101      4026532232   
1  129.051238        379       379                1     100      4026532231   
2  129.051434          1         1                0       0      4026531840   
3  129.051481          1         1                0       0      4026531840   
4  129.051522          1         1                0       0      4026531840   

       processName         hostName  eventId           eventName  \
0  systemd-resolve  ip-10-100-1-217       41              socket   
1  systemd-network  ip-10-100-1-217       41              socket   
2          systemd  ip-10-100-1-217     1005  security_file_open   
3          systemd  ip-10-100-1-217      257              openat   
4          systemd  ip-10-100-1-217        5               fstat   

                                      stackAddresses  argsNum  returnValue  \


In [ ]:
label_encoder = LabelEncoder()
for col in categorical_columns:
    data[col] = label_encoder.fit_transform(data[col])

X = data.drop(columns=['evil', 'stackAddresses', 'args', 'argsNum', 'returnValue'])
y = data['evil']

X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Shape of X after scaling:", X_scaled.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
from catboost import CatBoostClassifier

catboost_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    cat_features=[],
    verbose=100
)

# Train the model
catboost_model.fit(X_train, y_train)

# Predict on the test set
y_pred_cat = catboost_model.predict(X_test)
y_prob_cat = catboost_model.predict_proba(X_test)[:, 1]

# Evaluate the model using various metrics
accuracy_cat = accuracy_score(y_test, y_pred_cat)
precision_cat = precision_score(y_test, y_pred_cat)
recall_cat = recall_score(y_test, y_pred_cat)
f1_cat = f1_score(y_test, y_pred_cat)
cm_cat = confusion_matrix(y_test, y_pred_cat)

# Print evaluation metrics
print(f"CatBoost Model Evaluation:")
print(f"Accuracy: {accuracy_cat:.4f}")
print(f"Precision: {precision_cat:.4f}")
print(f"Recall: {recall_cat:.4f}")
print(f"F1 Score: {f1_cat:.4f}")


0:	learn: 0.3558461	total: 196ms	remaining: 3m 15s
100:	learn: 0.0002215	total: 18.8s	remaining: 2m 47s
200:	learn: 0.0001806	total: 35.1s	remaining: 2m 19s
300:	learn: 0.0001806	total: 51.1s	remaining: 1m 58s
400:	learn: 0.0001806	total: 1m 6s	remaining: 1m 40s
500:	learn: 0.0001806	total: 1m 24s	remaining: 1m 23s
600:	learn: 0.0001806	total: 1m 41s	remaining: 1m 7s
700:	learn: 0.0001806	total: 1m 58s	remaining: 50.6s
800:	learn: 0.0001806	total: 2m 14s	remaining: 33.4s
900:	learn: 0.0001806	total: 2m 30s	remaining: 16.5s
999:	learn: 0.0001806	total: 2m 47s	remaining: 0us
CatBoost Model Evaluation:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000


ImportError: Matplotlib requires numpy>=1.23; you have 1.22.4

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc, precision_recall_curve

# Confusion Matrix plot
plt.figure(figsize=(6, 4))
sns.heatmap(cm_cat, annot=True, fmt="d", cmap="Blues")
plt.title('CatBoost - Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC Curve
fpr_cat, tpr_cat, _ = roc_curve(y_test, y_prob_cat)
roc_auc_cat = auc(fpr_cat, tpr_cat)

# Precision-Recall Curve
precision_vals_cat, recall_vals_cat, _ = precision_recall_curve(y_test, y_prob_cat)
pr_auc_cat = auc(recall_vals_cat, precision_vals_cat)

# Plot ROC and Precision-Recall curves
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# ROC Curve
axes[0].plot(fpr_cat, tpr_cat, color='blue', lw=2, label=f"ROC Curve (AUC = {roc_auc_cat:.2f})")
axes[0].plot([0, 1], [0, 1], color='gray', linestyle='--')
axes[0].set_title('CatBoost - ROC Curve')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].legend(loc="lower right")

# Precision-Recall Curve
axes[1].plot(recall_vals_cat, precision_vals_cat, color='blue', lw=2, label=f"PR Curve (AUC = {pr_auc_cat:.2f})")
axes[1].set_title('CatBoost - Precision-Recall Curve')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].legend(loc="lower left")

plt.tight_layout()
plt.show()

In [ ]:
import joblib

joblib.dump(catboost_model, "xgboost.pkl")

joblib.dump(y_test, 'y_test_catboost.joblib')
joblib.dump(y_prob_cat, 'y_prob_catboost.joblib')

['y_prob_catboost.joblib']